# Práctica opcional Hadoop y Spark 

- Blanca Cano Camarero
- Iker Villegas Labairu



## Dataset 1: Análisis del comportamiento local de los diferentes equipos que han jugado en la Premier League inglesa  

### Descripción del dataset 

Puede encontrar los datos en [aquí](https://www.kaggle.com/datasets/zaeemnalla/premier-league), nos vamos a centrar en el fichero `results.csv` que cuenta con las siguientes 6 columnas: 

- `home_team`: String, nombre del equipo que juega en casa, hay 39 valores únicos. 

- `home_goals`: Number, número de goles marcados por el equipo local.

- `away_goals`:  Número de goles marcados por el equipo visitante.

- `results`: String que puede ser `H,A,D` he indica al ganador. `H` para el local, `A` para el visitante, `D` si hay empate. 


En total hay $4560$ datos donde todos los atributos son correctos (ie no contiene valores perdidos o mal clasificados). 

### Objetivo de la aplicación   

Se pretende saber cómo se comportan en local los equipos, es decir responder a las siguientes preguntas: 

1. ¿Cómo juega cada equipo en local en cada temporada?
2. ¿Existe una tendencia a ganar más jugando como local?

### Estructura de la solución propuesta

Los pasos a seguir son: 
- Se cargarán los datos desde google drive.
- No se realizará un preprocesamiento ya que todos los datos son correctos. 
- Con pyspark se realizará el tratamiento de los datos necesario para extraer la información. 




### Subida de datos. 

Se ha añadido a google drive el fichero de `resultados` (puede encontrarlo en la carpeta). 

y ahora vamos a proceder a leerlo instalando antes todos los paquetes necesarios como ya hicimos en la práctica de Spark. 


In [1]:
# instalar Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Descargar la ultima versión de java ( comprobar que existen los path de descarga)
!wget -q https://downloads.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz
# instalar pyspark
!pip install -q pyspark
# variables de entorno
import os # libreria de manejo del sistema operativo
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"
from pyspark.sql import SparkSession
# Iniciamos sesión con spark
APP_NAME = "PDGE-AnalisisFutbol"
SPARK_URL = "local[*]"
spark = SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()
spark

In [2]:
# Cargamos contexto de spark
sc = spark.sparkContext
#obtener el contexto de ejecución de Spark del Driver.
sc

<SparkContext master=local[*] appName=PDGE-AnalisisFutbol>

In [3]:
# Lectura del fichero desde google drive
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
# Nota: He tenido que añadir la ruta esta para concreta para que lo lea en mi drive
data = sc.textFile("gdrive/MyDrive/Colab\ Notebooks/results.csv")


In [5]:
data.take(10)

['home_team,away_team,home_goals,away_goals,result,season',
 'Sheffield United,Liverpool,1.0,1.0,D,2006-2007',
 'Arsenal,Aston Villa,1.0,1.0,D,2006-2007',
 'Everton,Watford,2.0,1.0,H,2006-2007',
 'Newcastle United,Wigan Athletic,2.0,1.0,H,2006-2007',
 'Portsmouth,Blackburn Rovers,3.0,0.0,H,2006-2007',
 'Reading,Middlesbrough,3.0,2.0,H,2006-2007',
 'West Ham United,Charlton Athletic,3.0,1.0,H,2006-2007',
 'Bolton Wanderers,Tottenham Hotspur,2.0,0.0,H,2006-2007',
 'Manchester United,Fulham,5.0,1.0,H,2006-2007']

Vamos a procesar los datos para que podamos trabajar con ellos. 

Para ello: 
1. Eliminamos primera fila
2. Realizamos un split elemento a elemento. 
Esto nos dará una lista de listas, donde para cada sublista los elementos serán: 

| Atributo | Índice | 
| --- | --- | 
 home_team | 0
away_team | 1
home_goals | 2
 away_goals | 3
 result | 4
  season |  5



In [6]:
data_without_head = data.filter(lambda x: "-" in x)
splited_data = data_without_head.map(lambda d: tuple(d.split(',')))
splited_data.take(5)

[('Sheffield United', 'Liverpool', '1.0', '1.0', 'D', '2006-2007'),
 ('Arsenal', 'Aston Villa', '1.0', '1.0', 'D', '2006-2007'),
 ('Everton', 'Watford', '2.0', '1.0', 'H', '2006-2007'),
 ('Newcastle United', 'Wigan Athletic', '2.0', '1.0', 'H', '2006-2007'),
 ('Portsmouth', 'Blackburn Rovers', '3.0', '0.0', 'H', '2006-2007')]

Con esto ya estamos preparados para realizar los primeros cálculos. 

Los equipos que más victorias tenga en local por años, para ello primero sacaremos los años distintos que hay. 



In [7]:
get_season = splited_data.map(lambda x: x[5]).distinct()
season_list = get_season.collect()
season_list.sort()
print(season_list)

['2006-2007', '2007-2008', '2008-2009', '2009-2010', '2010-2011', '2011-2012', '2012-2013', '2013-2014', '2014-2015', '2015-2016', '2016-2017', '2017-2018']


In [8]:
results = {}
# Solo si se ha ganado contará como una victoria en local
letter_to_victory = lambda x :  x == 'H' if 1 else 0 
for season in season_list:
  # 1. Filtramos los datos de cada temporada
  # 2. Mapeamos cada tupla para que solo quede (clave: equipo local, valor: 1 si ganó 0 en caso contrario) 
  filtered_season = splited_data.filter(lambda x: x[5]== season).map(lambda x : (x[0],letter_to_victory(x[4])))
  # 3. Sumamos las victorias de cada equipo
  reduced = filtered_season.reduceByKey(lambda a,b: a+b)
  results[season] = reduced.collect()
  results[season].sort(key= lambda x: -x[1]) # Los mejores resultados antes

#### Muestra de cada victoria. 

Como resultado de la operación tenemos un diccionario con cada una de las vistorias en local ordenadas 

In [9]:
top = 3 # número de mejores jugadores
for season in [season_list[2],season_list[5], season_list[-3], season_list[-1]]:
  print(f'\nLos {top} mejores equipos en {season} fueron:')
  equipos = results[season][:top]
  [print(f"{i+1}. {equipos[i][0]} con {equipos[i][1]} victorias locales") for i in range(0,top)]


Los 3 mejores equipos en 2008-2009 fueron:
1. Manchester United con 16 victorias locales
2. Manchester City con 13 victorias locales
3. Liverpool con 12 victorias locales

Los 3 mejores equipos en 2011-2012 fueron:
1. Manchester City con 18 victorias locales
2. Manchester United con 15 victorias locales
3. Tottenham Hotspur con 13 victorias locales

Los 3 mejores equipos en 2015-2016 fueron:
1. Manchester United con 12 victorias locales
2. Manchester City con 12 victorias locales
3. Leicester City con 12 victorias locales

Los 3 mejores equipos en 2017-2018 fueron:
1. Manchester City con 16 victorias locales
2. Manchester United con 15 victorias locales
3. Arsenal con 15 victorias locales


In [10]:
# Vamos a mostrar los resultados totales
#!pip install -q tabulate # para imprimir tablas bonitas
from tabulate import tabulate 

cabecera = ['Posición', 'Equipo', 'Victorias locales']
for season in season_list:
  equipos = results[season]
  tabla = [ [i+1, e[0], e[1]]for i,e in enumerate(equipos)]
  print(f"\n\n## Resultados temporada {season}  \n")
  print(tabulate(tabla, cabecera, tablefmt="github"))



## Resultados temporada 2006-2007  

|   Posición | Equipo            |   Victorias locales |
|------------|-------------------|---------------------|
|          1 | Manchester United |                  15 |
|          2 | Liverpool         |                  14 |
|          3 | Chelsea           |                  12 |
|          4 | Arsenal           |                  12 |
|          5 | Tottenham Hotspur |                  12 |
|          6 | Reading           |                  11 |
|          7 | Everton           |                  11 |
|          8 | Portsmouth        |                  11 |
|          9 | Middlesbrough     |                  10 |
|         10 | Bolton Wanderers  |                   9 |
|         11 | Blackburn Rovers  |                   9 |
|         12 | West Ham United   |                   8 |
|         13 | Newcastle United  |                   7 |
|         14 | Aston Villa       |                   7 |
|         15 | Fulham            |               

## La siguiente cuestión es si existe un beneficio a jugar en local.   

Para ello analizaremos el comportamiento de un equipo (introducir su nombre en la variable `equipo`) y después el de todas los equipos.

Formulación del experimento: 

Realizaremos un [Test de los rangos con signos de Wilcoxon](https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test) ya que es el que más se adecua a la situación, 
donde 
$$Z = \text{victorias locales} - \text{victorias como visitante}$$
para cada temporada.
- La hipótesis nula $H_0$: $Z$ sigue una distribución de media $0$. 
- La hipótesis alternativa $H_1$: $Z$ **no** sigue una distribución de media 0. 

Vamos a proceder a calcular la variable $Z$ para un equipo concreto: 

### Descripción del tratamiento de los datos. 

Se pretende calcular la variable $z$, esto es la diferencia entre victorias como local y como visitante para un cierto equipo en una cierta temporada, para ello:

1. Se filtran los datos para quedarnos solo con los concernientes al equipo buscado. 
2. Se transformarán los datos obtenidos en (1) en par `(temporada, valor)` la temporada es un `str`y valor es un entero $-1,1,0$. que indica respectivamente si ganó como visitante, como local o ninguno de esos casos. 
3. Se suman todos los datos de la misma temporada y el resultado es una lista con $z$ por temporada. 

In [11]:
equipo = 'Manchester United'

def traductor(x:tuple,equipo:str):
  '''Devuelve:
  1 si el equipo gana como local
  -1 si el equipo gana como visitante
  0 en caso contrario
  '''
  es_local = x[0] == equipo
  salida = 0 
  # Si gana como local
  if es_local and x[4] == 'H':
    salida = 1
  # Si gana como visitante
  if not es_local and x[4] == 'A':
    salida = -1
  return salida

def victorias_locales_menos_visitante_por_equipo(equipo):
  # Nos quedamos solo donde juegue el equipo 
  partidos_relevantes = splited_data.filter(lambda x: x[0]== equipo or x[1] == equipo)
  # nos quedamos con las victorias locales por sesión 
  victorias_locales = partidos_relevantes.map(lambda x:(x[5],traductor(x, equipo)))
  Z = victorias_locales.reduceByKey(lambda x,y: x+y).map(lambda x: x[1])
  z_list = Z.collect()
  return z_list

print(f"La diferencia entre las victorias locales y como visitante del {equipo} son: ")
print(victorias_locales_menos_visitante_por_equipo(equipo))

La diferencia entre las victorias locales y como visitante del Manchester United son: 
[8, -2, 5, 2, 7, 4, 5, 13, 2, 4, -1, 5]


Vamos a proceder a calcular el test, para ello utilizaremos la biblioteca de [scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.wilcoxon.html)

In [12]:
from scipy.stats import wilcoxon

def test_wilcoxon(equipo, a=0.05): 
  res = wilcoxon(victorias_locales_menos_visitante_por_equipo(equipo))
  if res.pvalue < a :
    print(f"Se ha rechazado la hipótesis nula con un p-valor de {res.pvalue} para el equipo {equipo}")
  else:
    print(f"No se ha podido rechazar la hipótesis nula ya que el p-valor es de {res.pvalue} para el equipo {equipo}")

In [13]:
# Vamos a mostrar algunos ejemplos concretos
equipos = ['Arsenal', 'Liverpool', 'Portsmouth', 'Manchester United']
for equipo in equipos:
  test_wilcoxon(equipo)

Se ha rechazado la hipótesis nula con un p-valor de 0.00048828125 para el equipo Arsenal


/usr/local/lib/python3.7/dist-packages/scipy/stats/morestats.py:3141: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


Se ha rechazado la hipótesis nula con un p-valor de 0.010799590882009212 para el equipo Liverpool
No se ha podido rechazar la hipótesis nula ya que el p-valor es de 0.25 para el equipo Portsmouth
Se ha rechazado la hipótesis nula con un p-valor de 0.00341796875 para el equipo Manchester United


Cabe destacar que el equipo Porsmouth descendió de división y que por tanto se tienen poco datos para su análisis, este puede ser el causante de que se pueda recharzar la hipótesis nula.  

### Diseño del tratamiento de los datos 

Vamos a ver cuáles son los equipos y el número de temporadas en las que estuvieron, para ellos:  

1. Transformamos cada tupla a un par `(equipo, temporada)`.
2. Se eliminal las filas redundantes. 
3. Sumamos cada aparición de cada equipo.



In [14]:
team_and_season = splited_data.map(lambda x: (x[0],x[5])).distinct()
team_appears = team_and_season.map(lambda x:(x[0],1)).reduceByKey(lambda x,y:x+y)
teams = team_appears.collect()
teams.sort(key=lambda x: -x[1])
## Veamos cuales son los equipos que han jugado un mínimo de temporadas
temporadas_jugadas_minimas = 11 
equipos_persistentes = team_appears.filter(lambda x: x[1] >= temporadas_jugadas_minimas).map(lambda x: x[0]).collect()

Vamos a realizar el experimento para todos los equipos que han jugado más de `temporadas_jugadas_minimas`.

In [15]:
print(f'Los equipos que han jugado más de {temporadas_jugadas_minimas} son: \n{equipos_persistentes}')
print('Veamos si estos equipos tienen más a ganar en local que en visitante')
for equipo in equipos_persistentes:
  test_wilcoxon(equipo)

Los equipos que han jugado más de 11 son: 
['Manchester United', 'Chelsea', 'Manchester City', 'Arsenal', 'Everton', 'West Ham United', 'Tottenham Hotspur', 'Liverpool']
Veamos si estos equipos tienen más a ganar en local que en visitante
Se ha rechazado la hipótesis nula con un p-valor de 0.00341796875 para el equipo Manchester United
Se ha rechazado la hipótesis nula con un p-valor de 0.04046183578416871 para el equipo Chelsea
Se ha rechazado la hipótesis nula con un p-valor de 0.007466958429676204 para el equipo Manchester City
Se ha rechazado la hipótesis nula con un p-valor de 0.00048828125 para el equipo Arsenal
Se ha rechazado la hipótesis nula con un p-valor de 0.00146484375 para el equipo Everton
Se ha rechazado la hipótesis nula con un p-valor de 0.0009765625 para el equipo West Ham United
Se ha rechazado la hipótesis nula con un p-valor de 0.00048828125 para el equipo Tottenham Hotspur
Se ha rechazado la hipótesis nula con un p-valor de 0.010799590882009212 para el equipo Li

Como vemos todos los equipos que han jugado más de 11 temporadas tienden a ganar con más partidos como locales que como visitantes. 


## Experimento 2 beneficio de ganar en local más para todos los equipos

### Diseño tratamiento de los datos

Se busca como objetivo calcular la diferencia a ganar como local o como visitante de los equipos independientemente de la temporada. 

1. Filtramos los empates que en nuestro caso no aportan información. 
2. Transformamos los datos resultante en pares  `(equipo ganados, -1 o 1)` donde $-1$ corresponde si gana como visitante y $1$ si lo hace como local. 
3. Se suman todos los datos de un mismo equipo. 

In [16]:
def traductor_dos(x):
  '''
    La clave es el nombre del equipo y el año. 
  '''
  es_local = x[4] == 'H'
  if es_local:
    return x[0]+x[5], 1 # si gana como local sumamos 1 al equipo ese año
  else:
    return x[1]+x[5], -1  # si gana como visitante restamos uno al equipo ese año

def victorias_locales_menos_visitante():
  # Nos quedamos solo donde se gane a se pierda
  partidos_relevantes = splited_data.filter(lambda x: x[4] != 'D') 
  victorias_locales = partidos_relevantes.map(traductor_dos)
  Z = victorias_locales.reduceByKey(lambda x,y: x+y).map(lambda x: x[1])
  z_list = Z.collect()
  return z_list

a = 0.05
res = wilcoxon(victorias_locales_menos_visitante())
if res.pvalue < a :
  print(f"Se ha rechazado la hipótesis nula con un p-valor de {res.pvalue}.")
else:
    print(f"No se ha podido rechazar la hipótesis nula ya que el p-valor es de {res.pvalue}.")

Se ha rechazado la hipótesis nula con un p-valor de 2.0695824579590253e-34.


# Conclusiones 

Hemos extraído las victorias para cada equipo en cada temporada, que han sido: 

## Resultados temporada 2006-2007  

|   Posición | Equipo            |   Victorias locales |
|------------|-------------------|---------------------|
|          1 | Manchester United |                  15 |
|          2 | Liverpool         |                  14 |
|          3 | Chelsea           |                  12 |
|          4 | Arsenal           |                  12 |
|          5 | Tottenham Hotspur |                  12 |
|          6 | Reading           |                  11 |
|          7 | Everton           |                  11 |
|          8 | Portsmouth        |                  11 |
|          9 | Middlesbrough     |                  10 |
|         10 | Bolton Wanderers  |                   9 |
|         11 | Blackburn Rovers  |                   9 |
|         12 | West Ham United   |                   8 |
|         13 | Newcastle United  |                   7 |
|         14 | Aston Villa       |                   7 |
|         15 | Fulham            |                   7 |
|         16 | Sheffield United  |                   7 |
|         17 | Charlton Athletic |                   7 |
|         18 | Manchester City   |                   5 |
|         19 | Wigan Athletic    |                   5 |
|         20 | Watford           |                   3 |


## Resultados temporada 2007-2008  

|   Posición | Equipo            |   Victorias locales |
|------------|-------------------|---------------------|
|          1 | Manchester United |                  17 |
|          2 | Arsenal           |                  14 |
|          3 | Chelsea           |                  12 |
|          4 | Liverpool         |                  12 |
|          5 | Manchester City   |                  11 |
|          6 | Everton           |                  11 |
|          7 | Aston Villa       |                  10 |
|          8 | Sunderland        |                   9 |
|          9 | Reading           |                   8 |
|         10 | Wigan Athletic    |                   8 |
|         11 | Newcastle United  |                   8 |
|         12 | Tottenham Hotspur |                   8 |
|         13 | Blackburn Rovers  |                   8 |
|         14 | Bolton Wanderers  |                   7 |
|         15 | Middlesbrough     |                   7 |
|         16 | West Ham United   |                   7 |
|         17 | Portsmouth        |                   7 |
|         18 | Birmingham City   |                   6 |
|         19 | Fulham            |                   5 |
|         20 | Derby County      |                   1 |


## Resultados temporada 2008-2009  

|   Posición | Equipo               |   Victorias locales |
|------------|----------------------|---------------------|
|          1 | Manchester United    |                  16 |
|          2 | Manchester City      |                  13 |
|          3 | Liverpool            |                  12 |
|          4 | Chelsea              |                  11 |
|          5 | Fulham               |                  11 |
|          6 | Arsenal              |                  11 |
|          7 | Stoke City           |                  10 |
|          8 | Tottenham Hotspur    |                  10 |
|          9 | West Ham United      |                   9 |
|         10 | Wigan Athletic       |                   8 |
|         11 | Everton              |                   8 |
|         12 | Portsmouth           |                   8 |
|         13 | Bolton Wanderers     |                   7 |
|         14 | Aston Villa          |                   7 |
|         15 | West Bromwich Albion |                   7 |
|         16 | Sunderland           |                   6 |
|         17 | Blackburn Rovers     |                   6 |
|         18 | Middlesbrough        |                   5 |
|         19 | Newcastle United     |                   5 |
|         20 | Hull City            |                   3 |


## Resultados temporada 2009-2010  

|   Posición | Equipo                  |   Victorias locales |
|------------|-------------------------|---------------------|
|          1 | Chelsea                 |                  17 |
|          2 | Manchester United       |                  16 |
|          3 | Arsenal                 |                  15 |
|          4 | Tottenham Hotspur       |                  14 |
|          5 | Liverpool               |                  13 |
|          6 | Manchester City         |                  12 |
|          7 | Fulham                  |                  11 |
|          8 | Everton                 |                  11 |
|          9 | Blackburn Rovers        |                  10 |
|         10 | Sunderland              |                   9 |
|         11 | Aston Villa             |                   8 |
|         12 | Birmingham City         |                   8 |
|         13 | Burnley                 |                   7 |
|         14 | Stoke City              |                   7 |
|         15 | West Ham United         |                   7 |
|         16 | Bolton Wanderers        |                   6 |
|         17 | Wigan Athletic          |                   6 |
|         18 | Hull City               |                   6 |
|         19 | Wolverhampton Wanderers |                   5 |
|         20 | Portsmouth              |                   5 |


## Resultados temporada 2010-2011  

|   Posición | Equipo                  |   Victorias locales |
|------------|-------------------------|---------------------|
|          1 | Manchester United       |                  18 |
|          2 | Chelsea                 |                  14 |
|          3 | Manchester City         |                  13 |
|          4 | Liverpool               |                  12 |
|          5 | Arsenal                 |                  11 |
|          6 | Bolton Wanderers        |                  10 |
|          7 | Stoke City              |                  10 |
|          8 | Tottenham Hotspur       |                   9 |
|          9 | Everton                 |                   9 |
|         10 | Aston Villa             |                   8 |
|         11 | Wolverhampton Wanderers |                   8 |
|         12 | Fulham                  |                   8 |
|         13 | West Bromwich Albion    |                   8 |
|         14 | Blackburn Rovers        |                   7 |
|         15 | Sunderland              |                   7 |
|         16 | Newcastle United        |                   6 |
|         17 | Birmingham City         |                   6 |
|         18 | Wigan Athletic          |                   5 |
|         19 | Blackpool               |                   5 |
|         20 | West Ham United         |                   5 |


## Resultados temporada 2011-2012  

|   Posición | Equipo                  |   Victorias locales |
|------------|-------------------------|---------------------|
|          1 | Manchester City         |                  18 |
|          2 | Manchester United       |                  15 |
|          3 | Tottenham Hotspur       |                  13 |
|          4 | Chelsea                 |                  12 |
|          5 | Arsenal                 |                  12 |
|          6 | Newcastle United        |                  11 |
|          7 | Fulham                  |                  10 |
|          8 | Everton                 |                  10 |
|          9 | Swansea City            |                   8 |
|         10 | Queens Park Rangers     |                   7 |
|         11 | Stoke City              |                   7 |
|         12 | Sunderland              |                   7 |
|         13 | Norwich City            |                   7 |
|         14 | Blackburn Rovers        |                   6 |
|         15 | Liverpool               |                   6 |
|         16 | West Bromwich Albion    |                   6 |
|         17 | Wigan Athletic          |                   5 |
|         18 | Aston Villa             |                   4 |
|         19 | Bolton Wanderers        |                   4 |
|         20 | Wolverhampton Wanderers |                   3 |


## Resultados temporada 2012-2013  

|   Posición | Equipo               |   Victorias locales |
|------------|----------------------|---------------------|
|          1 | Manchester United    |                  16 |
|          2 | Manchester City      |                  14 |
|          3 | Chelsea              |                  12 |
|          4 | Everton              |                  12 |
|          5 | Arsenal              |                  11 |
|          6 | Tottenham Hotspur    |                  11 |
|          7 | Newcastle United     |                   9 |
|          8 | West Bromwich Albion |                   9 |
|          9 | West Ham United      |                   9 |
|         10 | Liverpool            |                   9 |
|         11 | Norwich City         |                   8 |
|         12 | Fulham               |                   7 |
|         13 | Stoke City           |                   7 |
|         14 | Southampton          |                   6 |
|         15 | Swansea City         |                   6 |
|         16 | Aston Villa          |                   5 |
|         17 | Sunderland           |                   5 |
|         18 | Reading              |                   4 |
|         19 | Wigan Athletic       |                   4 |
|         20 | Queens Park Rangers  |                   2 |


## Resultados temporada 2013-2014  

|   Posición | Equipo               |   Victorias locales |
|------------|----------------------|---------------------|
|          1 | Manchester City      |                  17 |
|          2 | Liverpool            |                  16 |
|          3 | Chelsea              |                  15 |
|          4 | Arsenal              |                  13 |
|          5 | Everton              |                  13 |
|          6 | Tottenham Hotspur    |                  11 |
|          7 | Stoke City           |                  10 |
|          8 | Manchester United    |                   9 |
|          9 | Newcastle United     |                   8 |
|         10 | Southampton          |                   8 |
|         11 | Crystal Palace       |                   8 |
|         12 | Hull City            |                   7 |
|         13 | West Ham United      |                   7 |
|         14 | Aston Villa          |                   6 |
|         15 | Norwich City         |                   6 |
|         16 | Swansea City         |                   6 |
|         17 | Fulham               |                   5 |
|         18 | Sunderland           |                   5 |
|         19 | Cardiff City         |                   5 |
|         20 | West Bromwich Albion |                   4 |


## Resultados temporada 2014-2015  

|   Posición | Equipo               |   Victorias locales |
|------------|----------------------|---------------------|
|          1 | Chelsea              |                  15 |
|          2 | Manchester United    |                  14 |
|          3 | Manchester City      |                  14 |
|          4 | Arsenal              |                  12 |
|          5 | Southampton          |                  11 |
|          6 | Stoke City           |                  10 |
|          7 | Liverpool            |                  10 |
|          8 | Tottenham Hotspur    |                  10 |
|          9 | West Ham United      |                   9 |
|         10 | Swansea City         |                   9 |
|         11 | Newcastle United     |                   7 |
|         12 | Leicester City       |                   7 |
|         13 | West Bromwich Albion |                   7 |
|         14 | Everton              |                   7 |
|         15 | Queens Park Rangers  |                   6 |
|         16 | Crystal Palace       |                   6 |
|         17 | Aston Villa          |                   5 |
|         18 | Hull City            |                   5 |
|         19 | Burnley              |                   4 |
|         20 | Sunderland           |                   4 |


## Resultados temporada 2015-2016  

|   Posición | Equipo               |   Victorias locales |
|------------|----------------------|---------------------|
|          1 | Manchester United    |                  12 |
|          2 | Manchester City      |                  12 |
|          3 | Leicester City       |                  12 |
|          4 | Arsenal              |                  12 |
|          5 | Southampton          |                  11 |
|          6 | Tottenham Hotspur    |                  10 |
|          7 | West Ham United      |                   9 |
|          8 | Stoke City           |                   8 |
|          9 | Swansea City         |                   8 |
|         10 | Liverpool            |                   8 |
|         11 | Newcastle United     |                   7 |
|         12 | Everton              |                   6 |
|         13 | Norwich City         |                   6 |
|         14 | West Bromwich Albion |                   6 |
|         15 | Sunderland           |                   6 |
|         16 | Watford              |                   6 |
|         17 | Crystal Palace       |                   6 |
|         18 | Chelsea              |                   5 |
|         19 | AFC Bournemouth      |                   5 |
|         20 | Aston Villa          |                   2 |


## Resultados temporada 2016-2017  

|   Posición | Equipo               |   Victorias locales |
|------------|----------------------|---------------------|
|          1 | Chelsea              |                  17 |
|          2 | Tottenham Hotspur    |                  17 |
|          3 | Arsenal              |                  14 |
|          4 | Everton              |                  13 |
|          5 | Liverpool            |                  12 |
|          6 | Manchester City      |                  11 |
|          7 | Burnley              |                  10 |
|          8 | Leicester City       |                  10 |
|          9 | AFC Bournemouth      |                   9 |
|         10 | West Bromwich Albion |                   9 |
|         11 | Hull City            |                   8 |
|         12 | Manchester United    |                   8 |
|         13 | Swansea City         |                   8 |
|         14 | Watford              |                   8 |
|         15 | Stoke City           |                   7 |
|         16 | West Ham United      |                   7 |
|         17 | Southampton          |                   6 |
|         18 | Crystal Palace       |                   6 |
|         19 | Middlesbrough        |                   4 |
|         20 | Sunderland           |                   3 |


## Resultados temporada 2017-2018  

|   Posición | Equipo                   |   Victorias locales |
|------------|--------------------------|---------------------|
|          1 | Manchester City          |                  16 |
|          2 | Manchester United        |                  15 |
|          3 | Arsenal                  |                  15 |
|          4 | Tottenham Hotspur        |                  13 |
|          5 | Liverpool                |                  12 |
|          6 | Chelsea                  |                  11 |
|          7 | Everton                  |                  10 |
|          8 | Newcastle United         |                   8 |
|          9 | Burnley                  |                   7 |
|         10 | Watford                  |                   7 |
|         11 | Crystal Palace           |                   7 |
|         12 | Brighton and Hove Albion |                   7 |
|         13 | AFC Bournemouth          |                   7 |
|         14 | Leicester City           |                   7 |
|         15 | West Ham United          |                   7 |
|         16 | Huddersfield Town        |                   6 |
|         17 | Swansea City             |                   6 |
|         18 | Stoke City               |                   5 |
|         19 | Southampton              |                   4 |
|         20 | West Bromwich Albion     |                   3 |


Además para los equipos de los que se tienen datos suficientes se ha encontrado que con más de un 95% de confianza estos equipos tienden a ganar más como locales que como visitantes. 

Finalmente se ha comprobado que de manera global este resultado también se ve avalado. 


## Conclusiones sobre Spark 

Nótese que para los distintos apartados de la práctica hemos debido de extraer los siguientes datos: 

- Victorias como local por equipo.
- El número de temporadas que juega cada equipo.
- Los equipos que juegan más de cierta cantidad de temporadas. 
- Diferencias de victorias como locales y como visitantes de cierto equipo por temporadas. 
- Diferencias de victorias como locales y como visitantes de todo los equipos por temporadas.

A pesar de ser python un lenguaje orientado a objetos la redacción de éstas tareas se han hecho con herramientas funcioneles como: `reduce`, `map`, `filter` y `groupByKey` y en muy pocas líneas, lo cual demuestra la expresividad de tan lenguaje. 